Nama : Faqih Syamil

NIM  : 162112133006

# Import

In [3]:
!pip install emoji
!pip install gensim==4.2.0
!pip install texthero==1.0.5
!pip install "gensim==3.8.1"

  Using cached gensim-4.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.0 MB)
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.2
    Uninstalling gensim-4.3.2:
      Successfully uninstalled gensim-4.3.2
  Using cached texthero-1.0.5-py3-none-any.whl (12 kB)
  Attempting uninstall: texthero
    Found existing installation: texthero 1.0.9
    Uninstalling texthero-1.0.9:
      Successfully uninstalled texthero-1.0.9
  Using cached gensim-3.8.1-cp310-cp310-linux_x86_64.whl
  Attempting uninstall: gensim
    Found existing installation: gensim 4.2.0
    Uninstalling gensim-4.2.0:
      Successfully uninstalled gensim-4.2.0


In [4]:
!pip install Unidecode

In [5]:
import numpy as np
import pandas as pd
import re
import emoji
import string
from nltk.corpus import wordnet
import texthero as hero

In [34]:
train = pd.read_csv('traindata.csv',sep=',')
test = pd.read_csv('testdata.csv',sep=',')
solution = pd.read_csv('sample_submission.csv',usecols=['Id','Category'])
solution = solution.loc[solution['Id'].isin(test['Id'])]

# Preprocessing

In [35]:
#menghapus duplicate data
train.drop_duplicates(subset='Text Tweet',inplace=True)

### Data Leakage

In [36]:
train[train['Text Tweet'].apply(len) < 10]

,Id,Sentiment,Pasangan Calon,Text Tweet
180,209,positive,Agus-Sylvi,#ahy


In [37]:
train.drop(180, inplace=True)

In [38]:
# Mengecek Simbol Tanda Baca
def check_tanda_baca(text):
    # Menggunakan translate() dan maketrans() untuk menghapus tanda baca
    translator = str.maketrans('', '', string.punctuation)
    text_without_punctuation = text.translate(translator)

    # Mengembalikan tanda baca yang dihapus
    punctuation = ''.join(filter(lambda x: x in string.punctuation, text))

    return punctuation

train['Text Tweet'].apply(check_tanda_baca)

0                     ##..
1                     #...
2                   @..#..
3                      ###
4                   #..#@@
              ...         
745    ,.!<>#.##://..//-_/
746               @@,#<>##
747               #<>###<>
748             <><>://.##
749                :%####%
Name: Text Tweet, Length: 748, dtype: object

In [39]:
# Mengecek Simbol Emoji
index_train_emoji = []
index_test_emoji = []

#cek train
for idx, review in enumerate(train['Text Tweet']):
    if any(char in emoji.EMOJI_DATA for char in review):
        index_train_emoji.append(idx)

#cek test
for idx, review in enumerate(test['Text Tweet']):
    if any(char in emoji.EMOJI_DATA for char in review):
        index_test_emoji.append(idx)

In [40]:
percentage_train_emoji= round(len(index_train_emoji) / train.shape[0] * 100, 2)
print(f'Train data memiliki {len(index_train_emoji)} baris yang menggunakan emoji, berarti {percentage_train_emoji} persen dari total')

percentage_test_emoji = round(len(index_test_emoji) / test.shape[0] * 100, 2)
print(f'Test data memiliki {len(index_test_emoji)} baris yang menggunakan emoji, berarti {percentage_test_emoji} persen dari total')

Train data memiliki 0 baris yang menggunakan emoji, berarti 0.0 persen dari total
Test data memiliki 0 baris yang menggunakan emoji, berarti 0.0 persen dari total


### Pipeline Clean

In [41]:
def delete_repeated_char(text):
    text = re.sub(r'(\w)\1{2,}', r'\1', text)
    return text

def remove_punc(review):
    review =  review.translate(str.maketrans('', '', string.punctuation))
    review = ' '.join([word for word in review.split()])
    review = review.lower()
    return review

def remove_punctuation_except(text, keep_chars):
    # Membuat daftar karakter tanda baca yang ingin dipertahankan
    keep_punctuation = ''.join(char for char in string.punctuation if char in keep_chars)
    # Menggunakan regex untuk menghapus tanda baca yang ingin dihapus
    cleaned_text = re.sub(f'[^{keep_punctuation}\w\s]', '', text)

    return cleaned_text

keep_chars = "?!#@"


In [42]:
train['Text'] = train['Text Tweet'].apply(delete_repeated_char)
test['Text'] = test['Text Tweet'].apply(delete_repeated_char)

In [43]:
# Normalisasi Teks
slang_words_indo = pd.read_csv("https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv")
slang_jadi_formal = dict(zip(slang_words_indo['slang'], slang_words_indo['formal']))
# Fungsi normalisasi teks
def normalize_text(text):
    words = text.split()  # Memecah teks menjadi kata-kata
    normalized_words = []
    for word in words:
        if word in slang_jadi_formal:
            normalized_words.append(slang_jadi_formal[word])
        else:
            normalized_words.append(word)
    normalized_text = ' '.join(normalized_words)  # Menggabungkan kata-kata kembali menjadi teks
    return normalized_text

# Mengaplikasikan normalisasi pada kolom 'Text' dalam DataFrame 'train'
train['Text'] = train['Text'].apply(normalize_text)
test['Text'] = test['Text'].apply(normalize_text)

In [44]:
NUM_TOP_WORDS = 5
train.groupby('Sentiment')['Text'].apply(lambda x: hero.top_words(x)[:NUM_TOP_WORDS])

Sentiment            
negative                 153
           AHY           113
           yang           82
           AhokDjarot     69
           Face           68
positive                 167
           AHY           120
           yang           99
           dan            85
           untuk          63
Name: Text, dtype: int64

In [45]:
# Feature Engineering
# encoding variabel Y menjadi 0 1
# 0 untuk sentimen negatif
# 1 untuk sentitment positif
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

train['Sentiment'] = label_encoder.fit_transform(train['Sentiment'])
solution['Category'] = label_encoder.transform(solution['Category'])

In [46]:
train

,Id,Sentiment,Pasangan Calon,Text Tweet,Text
0,1,0,Agus-Sylvi,Banyak akun kloning seolah2 pendukung #agussil...,Banyak akun kloning seolah2 pendukung #agussil...
1,2,0,Agus-Sylvi,#agussilvy bicara apa kasihan yaa...lap itu ai...,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,3,0,Agus-Sylvi,Kalau aku sih gak nunggu hasil akhir QC tp lag...,Kalau aku sih enggak menunggu hasil akhir QC t...
3,4,0,Agus-Sylvi,Kasian oh kasian dengan peluru 1milyar untuk t...,Kasian oh kasihan dengan peluru 1milyar untuk ...
4,5,0,Agus-Sylvi,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...
...,...,...,...,...,...
745,895,1,Anies-Sandi,"Ingat, pilih no.3 di tanggal 19 April nanti ya...","Ingat, pilih no.3 di tanggal 19 April nanti ya..."
746,896,1,Anies-Sandi,"Kali saja bpk @aniesbaswedan @sandiuno lihat, ...","Kali saja bpk @aniesbaswedan @sandiuno lihat, ..."
747,897,1,Anies-Sandi,Kita harus dapat merangkul semua orang tanpa b...,Kita harus dapat merangkul semua orang tanpa b...
748,898,1,Anies-Sandi,Ini jagoanku dibidang digital <Smiling Face Wi...,Ini jagoanku dibidang digital <Smiling Face Wi...


In [ ]:
solution

In [47]:
train.Sentiment.value_counts()

1    380
0    368
Name: Sentiment, dtype: int64

In [25]:
!pip install transformers

In [26]:
# BERT Transformer
from transformers import AutoTokenizer, AutoModelForMaskedLM
from tqdm import tqdm
tokenizer = AutoTokenizer.from_pretrained('indolem/indobertweet-base-uncased')
model = AutoModelForMaskedLM.from_pretrained("indolem/indobertweet-base-uncased")

Some weights of the model checkpoint at indolem/indobertweet-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
from tqdm import tqdm

def get_embeddings(texts, model, tokenizer, max_length=512):
    embeddings_x = []
    for tweet in tqdm(texts):
        encoded_input = tokenizer(tweet, return_tensors='pt', max_length=max_length, truncation=True)
        output = model(**encoded_input).logits
        emb_mean = output.mean(1)[0].detach().numpy()
        embeddings_x.append(emb_mean)
    return embeddings_x



embeddings_x_train = get_embeddings(train["Text Tweet"], model, tokenizer)
embeddings_x_test = get_embeddings(test["Text Tweet"], model, tokenizer)

100%|██████████| 150/150 [00:40<00:00,  3.71it/s]


# Model

In [28]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=632,n_jobs=-1,random_state=28)
rfc.fit(embeddings_x_train, train["Sentiment"])

RandomForestClassifier(n_estimators=632, n_jobs=-1, random_state=28)

In [29]:
y_test_pred = rfc.predict(embeddings_x_test)

In [30]:
y_test_pred

array([1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1])

# Evaluasi

In [49]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Akurasi
accuracy = accuracy_score(solution.Category.values, y_test_pred)
print(f"Akurasi: {accuracy:.2f}")

# F1-score
f1 = f1_score(solution.Category.values, y_test_pred)
print(f"F1-Score: {f1:.2f}")

# Presisi
precision = precision_score(solution.Category.values, y_test_pred)
print(f"Presisi: {precision:.2f}")

# Recall
recall = recall_score(solution.Category.values, y_test_pred)
print(f"Recall: {recall:.2f}")

# Matriks Konfusi
conf_matrix = confusion_matrix(solution.Category.values, y_test_pred)
print("Matriks Konfusi:")
print(conf_matrix)


Akurasi: 0.52
F1-Score: 0.64
Presisi: 0.82
Recall: 0.53
Matriks Konfusi:
[[13 14]
 [58 65]]


# Submissions

In [51]:
submission = pd.read_csv("sample_submission.csv")
submission["Category"] = y_test_pred
submission['Category'] = label_encoder.inverse_transform(submission['Category'])
submission.to_csv("FaqihSyamil_162112133006.csv", index=False)

In [52]:
submission

,Id,Category
0,807,positive
1,189,positive
2,223,positive
3,408,negative
4,8,positive
...,...,...
145,44,negative
146,747,negative
147,541,positive
148,203,positive


import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(submission, *['Id'], **{})
chart

import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(submission, *['Category'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(submission, *['Id'], **{})
chart

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(submission, *['Id', 'Category'], **{'inner': 'stick'})
chart

xgboost

In [57]:
from xgboost import XGBClassifier

# XGBoost
xgb = XGBClassifier(n_estimators=100, random_state=28, n_jobs=-1)
# Melatih model
xgb.fit(embeddings_x_train, train["Sentiment"])
# Membuat prediksi
y_test_pred_xgb = xgb.predict(embeddings_x_test)

In [58]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# Akurasi
accuracy_xgb = accuracy_score(solution.Category.values, y_test_pred_xgb)
print(f"Akurasi: {accuracy_xgb:.2f}")
# F1-score
f1_xgb = f1_score(solution.Category.values, y_test_pred_xgb)
print(f"F1-Score: {f1_xgb:.2f}")
# Presisi
presisi_xgb = precision_score(solution.Category.values, y_test_pred_xgb)
print(f"Presisi: {presisi_xgb:.2f}")
# Recall
recall_xgb = recall_score(solution.Category.values, y_test_pred_xgb)
print(f"Recall: {recall_xgb:.2f}")

# Matriks Konfusi
conf_matrix_xgb = confusion_matrix(solution.Category.values, y_test_pred_xgb)
print("Matriks Konfusi:")
print(conf_matrix_xgb)


Akurasi: 0.51
F1-Score: 0.63
Presisi: 0.83
Recall: 0.51
Matriks Konfusi:
[[14 13]
 [60 63]]


In [59]:
submission = pd.read_csv("sample_submission.csv")
submission["Category"] = y_test_pred_xgb
submission['Category'] = label_encoder.inverse_transform(submission['Category'])
submission.to_csv("submission6.csv", index=False)